<a href="https://colab.research.google.com/github/vijayrgopu/apache_beam/blob/master/10_Bank_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving cards.txt.gz to cards.txt.gz


In [0]:
!gzip -d loan.txt.gz
!gzip -d cards.txt.gz

In [0]:
!pip install --quiet apache_beam[gcp]

     |████████████████████████████████| 3.4MB 4.9MB/s 
     |████████████████████████████████| 59.2MB 67kB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 1.2MB 49.7MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 225kB 49.7MB/s 
     |████████████████████████████████| 143kB 48.5MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 235kB 38.8MB/s 
     |████████████████████████████████| 174kB 53.9MB/s 
     |████████████████████████████████| 122kB 48.3MB/s 
     |████████████████████████████████| 112kB 54.2MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: google-api-python-client 1.7.12 has requirement httplib2<1dev,>=0.17.0, but you'll have httplib2 0.12.0 which is incompatible.
ERROR: chainer 6.5.0 has requir

In [0]:
import apache_beam as beam
from datetime import datetime

In [0]:
def calculate_points(element):
  cust_id, f_name, l_name, relation, card_type, max_credit, t_spent, withdrawn, cleared, last_date = element.split(',')
  defaulter_points = 0
  key = cust_id + '-' + f_name + ' ' + l_name
  rule_1, rule_2 = False, False
  
  #70 percent business rule
  cust_70_perc = float(max_credit) * 0.7
  if(cust_70_perc >= float(cleared)):
    defaulter_points += 1
    rule_1 = True

  #full amount uncleared
  if(max_credit == t_spent and float(max_credit) > float(cleared)):
    defaulter_points += 1
    rule_2 = True

  #additional points
  if(rule_1 and rule_2):
    defaulter_points += 1  

  return key, defaulter_points

def find_ploan_defaulters(element):
  allowed_miss_payments = 4
  allowed_cons_payments = 2
  key, months_list = element
  months_list.sort()
  sorted_months = months_list
  #print(sorted_months)
  
  tot_payments =len(sorted_months)
  #return element
  
  #auto return when total payments < 8
  if (tot_payments <= (12 - allowed_miss_payments)):
    return key,tot_payments
  #auto return when total payments == 12  
  if (tot_payments == 12):
    return key,0  
  
  next_month = 1
  cons_miss = 0
  for month in sorted_months:
    if (int(next_month) != int(month)):
      #if (month - next_month > 2):
        #cons_miss += 1
      if (month - next_month >= 2):
        next_month = month + 1
        cons_miss += (month - next_month)
      if ((month - next_month) == 1):
        next_month = month + 1
        cons_miss += 1 
    else:
      next_month += 1

  if (cons_miss >= allowed_cons_payments):
      return key,cons_miss

  return key,0         

def append_months(element):
  payment_date = datetime.strptime(element[8].rstrip().lstrip(),'%d-%m-%Y')
  element.append(str(payment_date.month)) 
  return element

def count_late_payments(element):
  due_date = datetime.strptime(element[6].rstrip().lstrip(),'%d-%m-%Y')
  payment_date = datetime.strptime(element[8].rstrip().lstrip(),'%d-%m-%Y')
  
  if (payment_date > due_date):
    element.append(1)
  else:
    element.append(0)
  return element    

def format_output(element):
  key, missed_months = element
  return str(key) + ', ' + str(missed_months) + ' missed'

def return_tuple(element):
  thisTuple = element.split(',')
  return (thisTuple[0],thisTuple[1:])
  
with beam.Pipeline() as p1:
  cards_data = (
      p1
      |'Read from cards.txt' >> beam.io.ReadFromText('cards.txt',skip_header_lines=1)
      |'Calculate points' >> beam.Map(calculate_points)
      |'Sum Defaulter points' >> beam.CombinePerKey(sum)
      |'Filter in Defaulters' >> beam.Filter(lambda element : element[1] > 0)
      |'Format output' >> beam.Map(lambda for_mat : str(for_mat[0]+','+str(for_mat[1])))
      |'return tuple' >> beam.Map(return_tuple)
      #|'Write to defaulters files' >> beam.io.WriteToText('defaulters.txt')
  )

  p_loan_data = (
      p1
      |'Read from loan.txt' >> beam.io.ReadFromText('loan.txt',skip_header_lines=1)
      |'loan row split' >> beam.Map(lambda row : row.split(','))
      |'Filter per loan recs' >> beam.Filter(lambda element : element[5]=='Personal Loan')
      |'Append months' >> beam.Map(append_months)
      |'Find defaulters' >> beam.Map(lambda record : (record[0]+'-'+record[1]+' '+record[2],int(record[9])))
      |'Group by Key' >> beam.GroupByKey()
      |'Find Defaulters' >> beam.Map(find_ploan_defaulters)
      |'Filter Defaulters' >> beam.Filter(lambda element : element[1] > 0)
      |'Format personal loan output' >> beam.Map(format_output)
      #|'Write out' >> beam.io.WriteToText('out')
  )

  m_loan_data = (
      p1
      |'Read from loan.txt for meds' >> beam.io.ReadFromText('loan.txt',skip_header_lines=1)
      |'loan row split for med' >> beam.Map(lambda row : row.split(','))
      |'Filter per loan recs for med' >> beam.Filter(lambda element : element[5]=='Medical Loan')
      |'count m_loan miss payments' >> beam.Map(count_late_payments)
      |'prepare data' >> beam.Map(lambda elements : (elements[0]+'-'+elements[1]+' '+elements[2], int(elements[9])))
      |'group b m_loans by key' >> beam.CombinePerKey(sum)
      |'Filter out non missers' >> beam.Filter(lambda elements : elements[1] > 0)
      |'Format medical loan output' >> beam.Map(format_output)
  )

  loan_defaulters = (
      (p_loan_data,m_loan_data)
      |'Combine data' >> beam.Flatten()
      |'Tuple for loan' >> beam.Map(return_tuple)
  )

  combine_loan_card_defaulters = (
      {'card_defaulter':cards_data, 'loan_defaulter' : loan_defaulters}
      |beam.CoGroupByKey()
      |'Write Results' >> beam.io.WriteToText('out')
  )
  

In [0]:
!ls -ltr

total 20
drwxr-xr-x 1 root root  4096 Mar 18 16:23 sample_data
-rw-r--r-- 1 root root 13636 Apr  6 15:53 loan.txt.gz


In [0]:
!head -10 out-00000-of-00001

('CT28383-Miyako Burns', {'card_defaulter': [['9']], 'loan_defaulter': []})
('CT74474-Nanaho Brennan', {'card_defaulter': [['12']], 'loan_defaulter': [[' 7 missed']]})
('CT66322-Chris Bruce', {'card_defaulter': [['12']], 'loan_defaulter': [[' 4 missed']]})
('CT65528-Bonnie Barlow', {'card_defaulter': [['12']], 'loan_defaulter': []})
('CT84463-Isaac Bowman', {'card_defaulter': [['11']], 'loan_defaulter': []})
('CT12838-Isidore Albert', {'card_defaulter': [['12']], 'loan_defaulter': []})
('CT96185-Danielle Bradshaw', {'card_defaulter': [['12']], 'loan_defaulter': []})
('CT74827-Hanna Boyer', {'card_defaulter': [['12']], 'loan_defaulter': []})
('CT98239-Sayuri Case', {'card_defaulter': [['12']], 'loan_defaulter': []})
('CT57141-Kaori Brown', {'card_defaulter': [['12']], 'loan_defaulter': []})


In [0]:
list_def = []
with open('defaulters.txt-00000-of-00001') as file:
  for line in file:
    key,value = line.split(',')
    list_def.append((key,int(value)))

  list_def.sort(key=lambda x : x[1])
  for k,v in list_def:
    print(k,v,sep=' --> ')

In [0]:
!head -20 defaulters.txt-00000-of-00001